# ML-проекта "Сервис генерации изображений для дизайнеров"
## Описание продукта
Продукт - веб-сервис генерации изображений для дизайнеров, который использует искусственный интеллект для создания графических материалов в соответствии с визуальной политикой и бренд-буками. Он направлен на автоматизацию процесса дизайна с использованием комбинации моделей генерации изображений и мультимодальных LLM. Пользователь загружает входные данные - брендбук, требования к фирменному стилю, а также промпт того, что должно быть изображено. После начала генерации сервис делает следующее:
- улучшает исходный промпт;
- достает неструктурированную информацию из брендбуков и гайдбуков;
- генерирует изображения;
- оценивает полученный результат с точки зрения установленных правил и если он ниже установленного порога, то генерирует новый вариант пытаясь доработать промпт;
- предоставляет пользователю несколько вариантов изображений, которые можно выбрать или отредактировать.

Альтернативно можно использовать только валидатор для автоматической проверки изображений на предмет их соответствия брендбукам и гайдлайном. В таком случае пользователь загружает набор изображений для проверки и запускает валидатор.

## Работа над baseline моделью
Данный проект будет использовать существующие модели генерации изображений с помощью API. Основным отличием от существующих систем является анализ брендбуков и оценка сгенерированный изображений для их автоматического улучшения.

#### Предварительный анализ
Предварительный анализ показывает, что графические дизайнеры руководствуются 2 большими группами оценок:
- следование брендбуку. Это информация о цветах, шрифтах, об осообенностях работы с логотипом, дополнительные специфичные ограничения;
- общие подходы к графическому дизайну. Четкость визуальной иерархии, гармоничность композиции и баланса, эффективное использование пространства, внимание к деталям и т.д.

Создаваемый валидатор должен учитывать обе эти группы. В идеале выдавать 2 отдельные оценки: на сколько изображение соответствует брендбуку и на сколько в целом дизайн является "хорошим". Обе эти оценки являются композитными и варьируются от 0 до 1. Финальная оценка формируется из отдельных показателей с некоторым весом. Основная часть этих показателей формируется отдельными LLM агентами, то есть используется набор метрик G-eval. Стоит также учитывать, что в брендбуке могут быть произвольные требования на основе, которые потребуется формировать оценку динамически. 
Помимо оценок изображений с помощью LLM можно использовать алгоритмические подходы, например для цвета. Для этого возможно будем оценивать распределения цвета изображений с тем, что описано в брендбуке. Это может быть непростой задачей, так например изображение может содержать фотографию (которое не запрещено использовать брендбуком) и графические элементы с правильным набором цветов. Так как фотография занимает большую площадь, распределение цветов будет смещенным. Поэтому здесь надо будет как-то учитывать композицию.


### Сбор данных
Был собран набор данных с брендбуками.

В директории datasets/raw/brandbooks находится набор брендбуков разных компаний. Каждый из брендбуков содержит префикс для их идентификации. В директории datasets/raw/images/compliant находятся изображения, которые соответствуют брендбукам (префикс в названии).

#### Извлечение информации
Используя LLM извлечем ключевую информацию из брендбуков, которую будем в последствии использовать при разработке и отладки RAG при генерации изображений, а также реализации валидатора. 


In [1]:
import pdfplumber
import cv2
import itertools
import layoutparser as lp
import numpy as np
import seaborn as sns
import src.lib.color_analyzer as color_analyzer
import chromadb
import pandas as pd
from chromadb.utils import embedding_functions
from camel.agents import TaskSpecifyAgent, ChatAgent
from camel.responses.agent_responses import ChatAgentResponse
from camel.configs import QwenConfig, GeminiConfig
from camel.models import ModelFactory
from camel.messages import BaseMessage
from camel.types import ModelPlatformType
from PIL import Image
from src.lib.brand_dataset import BrandDataset
from src.lib.pdf_parser import PDFParser
from src.lib.settings import settings
from src.lib.prompts import JinjaPromptManager
from src.lib.models import BrandbookRules, ClassifiedRules, BrandbookInfo, LLMColorFontResponse, LLMResponsePrompt
from src.lib.openai_image_generator import OpenAIImageGenerator
from pathlib import Path
from io import BytesIO
from PIL import Image
from typing import Type
from sentence_transformers import SentenceTransformer



/home/user/anaconda3/envs/mfdp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


prompts_manager = JinjaPromptManager(settings.prompt_directory)
reasoning_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
        model_type="o4-mini-high",
        url=str(settings.openai_api_url),
        api_key=settings.token,
        model_config_dict={"temperature": 0.0, "max_tokens": 128000},
    )

multimodal_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="o4-mini-high",
    url=str(settings.openai_api_url),
    api_key=settings.token,
    model_config_dict={"temperature": 0.0, "max_tokens": 128000},
)

enhance_model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
    model_type="claude-3.7-sonnet:thinking",
    url=str(settings.openai_api_url),
    api_key=settings.token,
    model_config_dict={"temperature": 0.0, "max_tokens": 128000},
)

multimodal_agent = ChatAgent(
    model=multimodal_model,
    output_language="Russian"
)

thinking_agent = ChatAgent(
    model=reasoning_model,
    output_language="Russian"
)

enhance_agent = ChatAgent(
    model=enhance_model,
    output_language="Russian"
)

dataset = BrandDataset(Path("datasets"))
dataset_frame = dataset.get_dataframe()

Будем использовать pdfplumber для экстракции данных. Библиотека camel-ai не может нативно использовать pdf, только изображения и текст.
Кроме того, после первой обработки данных выяснилось, что стандартные механизмы pdfplumber не дают достаточно точных результатов, поэтому будем использовать layoutparser для более сложной обработки. LayoutParser с использованием модели детектирует логические блоки элементов, в частности группы текста. С помощью этой детекции улучшается процесс извлечения данных.
Для этих целей был разработан модель pdf_parser.

Для изъятия информации об использованных цветах можно использовать комбинированный подход:
1. Проитерироваться по всем страницам PDF документа. Извлечь текстовую информацию с учетом логической структуры (layout).
2. С помощью LLM уточнить на каких страницах находится информация о цветах.
3. Извлечь цвета с помощью мультимодальности.

Альтернативным вариантом можно оценить какие цвета используются в брендбуки, оценив частоту появления цветов за исключением белого и черного. Для этого был создан модуль color_analyzer.

In [ ]:
def analyze_image(page_image: Image.Image, 
                  agent: ChatAgent, 
                  prompt: str,
                  response_format: Type) -> ChatAgentResponse:
    buf = BytesIO()
    thumbnail_image = page_image.copy()
    thumbnail_image.thumbnail((1024, 1024))
    thumbnail_image.save(buf, format="PNG")
    buf.seek(0)
    png_img = Image.open(buf)
    agent.reset()
    message = BaseMessage.make_user_message(
        role_name="User", 
        content=prompt,
        image_list=[png_img])
    response = agent.step(message, response_format)
    return response.msgs[0].parsed


embedding_model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

parser = PDFParser(model_name="./model_final.pth", model_config_file_name="./config.yaml")


description_to_classes = {
    '''Правила использования логотипа компании и связанных с ним фирменных знаков. Включает требования к размерам логотипа, пропорциям, охранному полю и минимально допустимым размерам. Описывает варианты логотипа: основной, монохромный, инверсный, черно-белый. Регламентирует размещение логотипа на различных носителях, допустимые и недопустимые модификации. Содержит правила использования фирменного знака отдельно от логотипа, правила совместного размещения с другими логотипами. 
    Определяет правила масштабирования, позиционирования на различных форматах и фонах. Включает файловые форматы логотипа для различных целей использования.''': "logo_rules",

    '''Определение и стандартизация корпоративных цветов бренда. 
    Включает основную и дополнительную цветовые палитры с точными значениями в различных цветовых моделях: CMYK (для печати), RGB (для цифровых носителей), HEX (для веб), Pantone (для специальных красок). 
    Содержит правила использования цветов в различных ситуациях, допустимые цветовые сочетания и пропорции. Описывает правила применения цветов для различных элементов дизайна, фонов, текста. Регламентирует использование градиентов, оттенков и прозрачности. 
    Включает рекомендации по цветовому кодированию информации и адаптации цветов для различных носителей с учетом особенностей восприятия и технических ограничений.''' : "color_system_rules",

    '''Система правил использования шрифтов и текстовых элементов. Определяет основные и дополнительные корпоративные шрифты, их начертания (regular, bold, italic и др.). Устанавливает правила типографской иерархии: размеры шрифтов для заголовков разных уровней, подзаголовков, основного текста, подписей. Регламентирует интерлиньяж (межстрочное расстояние), кернинг (межбуквенное расстояние), трекинг. Содержит правила выравнивания текста, допустимые переносы, работу с абзацами. Описывает особенности типографики для различных носителей: печатных материалов, цифровых платформ, наружной рекламы. Включает рекомендации по контрасту текста и фона для обеспечения читаемости. 
    Определяет правила оформления списков, таблиц, цитат и других специальных текстовых элементов.''': "typography_rules",

    '''Четкие указания о недопустимых способах использования элементов фирменного стиля. Включает запреты на искажение пропорций логотипа, изменение фирменных цветов, использование неутвержденных шрифтов. Описывает недопустимые комбинации элементов, запрещенные фоны для размещения логотипа и текста. Регламентирует запреты на добавление эффектов к логотипу (тени, обводки, градиенты), если это не предусмотрено правилами. Содержит ограничения по минимальному размеру элементов для сохранения читаемости и узнаваемости. Определяет запреты на размещение логотипа рядом с неподходящим контентом или в контексте, противоречащем ценностям бренда. 
    Включает ограничения на использование устаревших версий элементов фирменного стиля и самостоятельное создание новых элементов.''': "restrictions_rules",

    '''Принципы и правила построения визуальных композиций с использованием элементов фирменного стиля. Включает рекомендации по расположению элементов на различных форматах, соблюдению баланса и пропорций. Описывает модульные сетки для верстки различных материалов: визиток, бланков, презентаций, рекламных макетов. Регламентирует правила компоновки текста и изображений, создания визуальной иерархии. Содержит принципы организации пространства, использования "воздуха" (пустого пространства). Определяет стилистические особенности фотографий и иллюстраций, их кадрирование и обработку. 
    Включает правила создания композиций для различных носителей с учетом их специфики: печатных материалов, цифровых платформ, наружной рекламы, упаковки, сувенирной продукции.''': "composition_rules",

    '''Регламентация процессов согласования и использования элементов фирменного стиля с юридической точки зрения. Включает порядок утверждения дизайн-макетов, ответственных лиц и сроки согласования. Описывает правила использования товарных знаков, знаков обслуживания и других объектов интеллектуальной собственности. Содержит информацию о правовой защите элементов фирменного стиля, правила указания знаков правовой охраны (®, ™, ©). Регламентирует порядок предоставления доступа к файлам фирменного стиля третьим лицам, лицензирование. Определяет правила использования фирменного стиля партнерами, дилерами, франчайзи. 
    Включает требования к оформлению договоров и соглашений, связанных с использованием фирменного стиля, ответственность за нарушение установленных правил. Включает правила утверждения и согласования.''': "legal_rules"
    
}


extracted_values = ()
use_llm_to_classify = False

for brandbook_path, frame in list(dataset_frame.groupby("brandbook_path"))[8:]:
    print(brandbook_path)
    brandbook_rules = BrandbookRules(rules=[])
    for index, (page, page_image) in enumerate(zip(parser.parse_pages(brandbook_path), 
                                                   parser.parse_pages_as_images(brandbook_path))):
        multimodal_agent.reset()

        response = multimodal_agent.step(prompts_manager.render_prompt("extract_brandbook_rules", 
                                                                {"page_information": page.to_markdown()}), 
                                                                BrandbookRules)
        rules_info: BrandbookRules = response.msgs[0].parsed

        visual_rules_info: BrandbookRules = analyze_image(page_image, multimodal_agent, 
                                                   prompts_manager.render_prompt("extract_brandbook_rules_from_image"),
                                                   BrandbookRules)
        
        response = multimodal_agent.step(prompts_manager.render_prompt("deduplicate_rules", {
            "text_rules": rules_info.rules_to_markdown(),
            "image_rules": visual_rules_info.rules_to_markdown()
        }), BrandbookRules)


        rules_info: BrandbookRules = response.msgs[0].parsed
        brandbook_rules.rules.extend(rules_info.rules)


    thinking_agent.reset()
    rules_as_markdown = brandbook_rules.rules_to_markdown()
    if use_llm_to_classify:
        # Правил может быть очень много и они могут не влезть в контекст на данном этапе. Оценим примерный размер.
        token_estimation = len(rules_as_markdown) / 3

        if token_estimation > 4096:
            print(f"Длина правил {token_estimation}. Разбиваем на чанки")
            # Разбиваем правила на строки и группируем в чанки подходящего размера. Так как правила в принципе не зависят друг от друга
            rules_lines = rules_as_markdown.split('\n')
            chunks = []
            current_chunk = []
            current_size = 0
            
            for line in rules_lines:
                line_tokens = len(line) / 3
                if current_size + line_tokens > 4000:  # Берем с запасом
                    chunks.append('\n'.join(current_chunk))
                    current_chunk = [line]
                    current_size = line_tokens
                else:
                    current_chunk.append(line)
                    current_size += line_tokens
                    
            if current_chunk:
                chunks.append('\n'.join(current_chunk))
            
            # Обрабатываем каждый чанк и объединяем результаты
            classified_rules = None
            
            for chunk in chunks:
                thinking_agent.reset()
                response = thinking_agent.step(prompts_manager.render_prompt("classify_rules", {
                    "rules": chunk
                }), ClassifiedRules)
                chunk_classified_rules = response.msgs[0].parsed
                
                if classified_rules is None:
                    classified_rules = chunk_classified_rules
                else:
                    # Объединяем результаты из разных чанков
                    for rule_list_name in ['color_system_rules', 'typography_rules', 'logo_rules', 
                                        'composition_rules', 'identity_rules', 'other_rules']:
                        if hasattr(classified_rules, rule_list_name) and hasattr(chunk_classified_rules, rule_list_name):
                            current_list = getattr(classified_rules, rule_list_name)
                            chunk_list = getattr(chunk_classified_rules, rule_list_name)
                            if isinstance(current_list, list) and isinstance(chunk_list, list):
                                current_list.extend(chunk_list)
        else: 
            response = thinking_agent.step(prompts_manager.render_prompt("classify_rules", {
                "rules": rules_as_markdown
            }), ClassifiedRules)
            classified_rules: ClassifiedRules = response.msgs[0].parsed
    else:
        # Используем embedding модели для классификации правил
        classified_rules_dict = {}
        descriptions = list(description_to_classes.keys())
        rules_embeddings = embedding_model.encode(brandbook_rules.rules)
        classes_embeddings = embedding_model.encode(descriptions)
        cosine_similarities_max = (rules_embeddings @ classes_embeddings.T).argmax(axis=1)
        for index, rule in enumerate(brandbook_rules.rules):
            class_index = cosine_similarities_max[index]
            class_name = description_to_classes[descriptions[class_index]]
            try:
                classified_rules_dict[class_name].append(rule)
            except KeyError:
                classified_rules_dict[class_name] = [rule]
        classified_rules = ClassifiedRules(**classified_rules_dict)

    thinking_agent.reset()
    response = thinking_agent.step(prompts_manager.render_prompt("struct_color_and_font_data",
                                                                 {
                                                                     "color_rules": classified_rules.color_system_rules_to_markdown(),
                                                                     "typography_rules": classified_rules.typography_rules_to_markdown()
                                                                 }), LLMColorFontResponse)
    colors_font_response: LLMColorFontResponse = response.msgs[0].parsed
    brandbook_info = BrandbookInfo(classified_rules=classified_rules, colors_rgb=colors_font_response.colors_rgb, 
                                   fonts=colors_font_response.fonts)
    brandbook_info.classified_rules = classified_rules
    
    brand_id = frame["brand_id"].iloc[0]
    processed_rules_path = Path("datasets") / "processed" / f"{brand_id}_extracted_rules.json"
    with open(processed_rules_path, 'w', encoding="utf8") as file:
        file.write(brandbook_info.model_dump_json(indent=2))
    

## Обработка и помещение правил в векторную базу для реализации RAG при генерации изображений

In [ ]:
dataset = BrandDataset(Path("datasets"))
embedding_model_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="jinaai/jina-embeddings-v3", trust_remote_code=True, model_kwargs={
    "default_task": "retrieval.passage"
}, device="cuda")
chroma_client = chromadb.Client()
dataset_dataframe = dataset.get_dataframe_with_info()
for brand_id, frame in dataset_dataframe.groupby("brand_id"):
    collection = chroma_client.get_or_create_collection(
        name=brand_id,
        embedding_function=embedding_model_function,
        configuration={
            "hnsw": {
                "space": "cosine"
            }
        }
    )
    first_row = frame.iloc[0]
    current_start = 0
    if first_row["classified_rules"]:
        for classification, rules in first_row["classified_rules"]:
            if rules:
                doc_ids = [f"id_{index}" for index in range(current_start, current_start + len(rules))]
                meta_datas = [{"class": classification} for index in range(len(rules))]
                collection.add(documents=rules, ids=doc_ids, embeddings=None, metadatas=meta_datas)
                current_start += len(rules)

## Сгенерируем изображения с использованием RAG

Проведем генерацию изображений по каждому брендбуку с учетом автоматического улучшения промпта. На первом этапе будем использовать только один провайдер для генерации изображений.

In [4]:
image_generator = OpenAIImageGenerator(api_key=settings.token,
                                       base_url=str(settings.openai_api_url),
                                       model="gpt-image-1",
                                       quality="high",
                                       moderation="low",
                                       size="1024x1024" )

In [ ]:
graphic_designer_prompts = ['''Необходима фотография иллюстрирующая образ гонки. На нем в верхней части должна быть надпись "Спорт это жизнь!"''', 
                            '''Нужен задний фон для слайда. Стилистика ар-деко''',
                            '''3d иллюстрация компании, которая действует в среде web3, агрегатор застройщиков и применяет искусственный интеллект. Композиция должна быть единая''']

queries_to_enhance = [
    "Какие основные цветы можно использовать?",
    "Что разрешено изображать?",
    "Какой шрифт использовать?",
    "Как должна выглядеть композиция?",
    "Что запрещено делать с изображением?"
]

generated_folder = Path("generated")
for brand_id, _ in list(dataset_dataframe.groupby("brand_id"))[2:]:
    print(brand_id)
    for index, initial_prompt in enumerate(graphic_designer_prompts):
        
        collection = chroma_client.get_collection(brand_id)
        current_prompt = initial_prompt
        print(current_prompt)
        for query in queries_to_enhance:
            task="retrieval.query"
            # Модель jina требуется особых промптов для оптимизации работы под разные задачи. API chromadb не дает возможность передавать тип задачи, поэтому приходится залезать в реализацию
            # и обращаться к скрытым полям (Или загружать embedding модель отдельно еще раз, что увеличит потребление памяти)
            query_embedding = embedding_model_function._model.encode(query, task=task, prompt_name=task)
            rules = collection.query(query_embeddings=query_embedding, n_results=10)
            rules_list = rules["documents"][0]
            rules_as_markdown = "- " + "\n- ".join(rules_list)
            enhance_agent.reset()

            response = enhance_agent.step(prompts_manager.render_prompt("enhance_initial_prompt", 
                                                                {"initial_prompt": current_prompt,
                                                                "query": query,
                                                                "rules": rules}), 
                                                                LLMResponsePrompt)
            enhanced_prompt: LLMResponsePrompt = response.msgs[0].parsed
            current_prompt = enhanced_prompt.prompt
            print(current_prompt)

        image = image_generator.generate_image(current_prompt)
        image.save(generated_folder / f"{brand_id}_prompt_{index}.png")

## Анализ цвета

Проанализируем цветовую составляющую сгенерированных изображений и попробуем на основе этой информации определить к какому брендбуку относится каждое из изображений.

In [3]:
def create_images_dataframe(generated_dir: Path):
    # Список для хранения данных
    data = []

    # Проверяем существование директории
    if not generated_dir.exists():
        print(f"Директория '{generated_dir}' не найдена")
        return pd.DataFrame()
    
    # Получаем список файлов в директории
    for file_path in generated_dir.iterdir():
        if file_path.suffix.lower() == ".png" and "_prompt_" in file_path.name:
            try:
                # Извлекаем идентификатор бренда (часть до "_prompt_")
                brand_id = file_path.name.split("_prompt_")[0]
                
                # Загружаем изображение
                img = Image.open(file_path)
                
                # Добавляем данные в список
                data.append({
                    "brand_id": brand_id,
                    "filename": file_path.name,
                    "image": img
                })
            except Exception as e:
                print(f"Ошибка при обработке файла {file_path.name}: {e}")
    
    # Создаем DataFrame
    df = pd.DataFrame(data)
    
    # Выводим информацию о DataFrame
    print(f"Всего изображений: {len(df)}")
    if not df.empty:
        display(df.head())
    
    return df

generated_folder = Path("generated")
generated_image_dataframe = create_images_dataframe(generated_folder)
dataset_frame = dataset.get_dataframe_with_info()

analysis = []
for brand_id, frame in dataset_frame.groupby("brand_id"):
    first_row = frame.iloc[0]
    brand_colors = [color.to_numpy() for color in first_row["colors_rgb"]]
    for index, row in generated_image_dataframe.iterrows():
        image_name = row["filename"]
        image = row["image"]
        dominant_colors = color_analyzer.extract_colors(image)
        dominant_colors = (color for color, freq in dominant_colors)
        distances = []
        temp_colors = []
        for brand_color, image_color in itertools.product(brand_colors, dominant_colors):
            distances.append(color_analyzer.color_distance_v2(brand_color, image_color))
            temp_colors.append(brand_color)
        distances = np.array(distances)
        analysis.append({
            "image_name": image_name,
            "minimal_distance": distances.min(),
            "closest_brandbook_color": temp_colors[distances.argmin()],
            "generated_for_brand": row["brand_id"],
            "compared_to_brand": brand_id
        })
color_analysis_frame =  pd.DataFrame(analysis)
color_analysis_frame.head()

Всего изображений: 42


,brand_id,filename,image
0,brand5,brand5_prompt_2.png,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1,brand10,brand10_prompt_1.png,<PIL.PngImagePlugin.PngImageFile image mode=RG...
2,brand8,brand8_prompt_0.png,<PIL.PngImagePlugin.PngImageFile image mode=RG...
3,brand13,brand13_prompt_1.png,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4,brand1,brand1_prompt_0.png,<PIL.PngImagePlugin.PngImageFile image mode=RG...


2025-05-17 16:53:53 - color_analyzer - INFO - Начало выполнения extract_colors
2025-05-17 16:53:53 - color_analyzer - INFO - После фильтрации осталось 473482 пикселей (45.2% от исходного количества)
2025-05-17 16:53:53 - color_analyzer - INFO - Завершение extract_colors, время выполнения: 0.11 сек.
2025-05-17 16:53:53 - color_analyzer - INFO - Начало выполнения extract_colors
2025-05-17 16:53:53 - color_analyzer - INFO - После фильтрации осталось 66472 пикселей (6.3% от исходного количества)
2025-05-17 16:53:53 - color_analyzer - INFO - Завершение extract_colors, время выполнения: 0.06 сек.
2025-05-17 16:53:53 - color_analyzer - INFO - Начало выполнения extract_colors
2025-05-17 16:53:54 - color_analyzer - INFO - После фильтрации осталось 1041528 пикселей (99.3% от исходного количества)
2025-05-17 16:53:54 - color_analyzer - INFO - Завершение extract_colors, время выполнения: 0.09 сек.
2025-05-17 16:53:54 - color_analyzer - INFO - Начало выполнения extract_colors
2025-05-17 16:53:54 - 

,image_name,minimal_distance,closest_brandbook_color,generated_for_brand,compared_to_brand
0,brand5_prompt_2.png,6.543571,"[0, 94, 227]",brand5,brand1
1,brand10_prompt_1.png,7.821185,"[255, 255, 255]",brand10,brand1
2,brand8_prompt_0.png,13.748961,"[255, 89, 90]",brand8,brand1
3,brand13_prompt_1.png,12.533327,"[0, 0, 0]",brand13,brand1
4,brand1_prompt_0.png,3.668089,"[88, 255, 138]",brand1,brand1


In [5]:
indexes = color_analysis_frame.groupby("image_name")["minimal_distance"].idxmin()
frame = color_analysis_frame.loc[indexes]
total_match = (frame["generated_for_brand"] == frame["compared_to_brand"]).sum()

print(f"Обнаружено совпадений: {total_match}")
print(f"Процент совпадений: {total_match / len(frame)}")

Обнаружено совпадений: 12
Процент совпадений: 0.2857142857142857


В ходе анализа цветов на основе частоты было выявлено следующее:
- основные цвета можно определить по пикселям брендбука, однако он будут немного смещенным, плюс палитра будет не полной. У брендбуков есть вспомогательные цвета, но они значительно меньше применяются, поэтому в палитре оказываются самые популярные.
- надежнее использовать именно LLM, которые понимают контекст для считывания данных.
- частотную характеристику цвета лучше использовать при валидации изображений. 
- расчет того, на сколько цвет изображения соответствует цвету брендбука будет осуществлять с помощью расчета дельта e (евклидово расстояние) в цветовом пространстве lab. Судя по данным из wiki, человеческий глаз воспринимает разницу между цветами при дельта e около 2.3.
- при изменении размера изображения происходит интерполяция пикселей и это может привести к изменению частоты цветов. Это следует учитывать при анализе цветов.
- базовый подход по анализу цветов для сгенерированный изображений смог правильно определить брендбук лишь в 28 процентах случаев. Способ расчета нужно улучшить. Например при анализе цветов изображений объединять цвета имеющие небольшую дельту е, тогда частота похожих цветов будет больше. 

## Заключение
В данном ноутбуке описан базовый подход по генерации изображений с учетом брендбука:
- постраничный анализ pdf брендбука с использованием комбинации мультимодальных LLM, а также классической модели для определения структуры документа и получением списка правил брендбука;
- сохранение правил брендбука в векторную базу данных;
- улучшение промпта на базе информации из векторной базы данных с помощью LLM;
- оценка цвета сгенерированной изображения;